In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import random
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Input, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import gc
import os

# Enable memory growth to prevent TensorFlow from allocating all GPU memory at once
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)

# Import the PRESENT cipher implementation
# Assumes the pypresent.py file is in the same directory
from pypresent import Present

class DifferentialCryptanalysisModel:
    def __init__(self, rounds=3):
        """
        Initialize the differential cryptanalysis model for PRESENT cipher
        
        Args:
            rounds: Number of rounds to analyze (reduced round version)
        """
        self.rounds = rounds
        self.present = None
        self.model = None
        self.batch_size = 128  # Optimized batch size for M2
        
    def generate_dataset(self, num_samples=5000, key_bits=128):
        """
        Generate dataset of plaintext-ciphertext pairs for training
        
        Args:
            num_samples: Number of samples to generate
            key_bits: Key size in bits (80 or 128)
        
        Returns:
            X: Input features (contains plaintext pairs with specific differences)
            y: Target values (output differences in specific bits)
        """
        # Generate a random key
        if key_bits == 80:
            key_bytes = 10
        else:
            key_bytes = 16
            
        key = bytes([random.randint(0, 255) for _ in range(key_bytes)])
        self.present = Present(key, rounds=self.rounds)
        
        # Generate data in smaller batches to avoid memory issues
        batch_size = 1000
        num_batches = num_samples // batch_size
        
        X_list = []
        y_list = []
        
        for batch in range(num_batches):
            # Arrays to store our batch data
            X_batch = np.zeros((batch_size, 128), dtype=np.float32)  # Using float32 for TensorFlow compatibility
            y_batch = np.zeros((batch_size, 64), dtype=np.float32)
            
            for i in range(batch_size):
                # Generate random plaintext
                plaintext1 = bytes([random.randint(0, 255) for _ in range(8)])
                
                # Create second plaintext with specific difference
                plaintext_bytes = bytearray(plaintext1)
                plaintext_bytes[0] ^= 0x01  # Introducing a difference in the MSB of first byte
                plaintext2 = bytes(plaintext_bytes)
                
                # Encrypt both plaintexts
                ciphertext1 = self.present.encrypt(plaintext1)
                ciphertext2 = self.present.encrypt(plaintext2)
                
                # Convert to bit arrays
                pt1_bits = self._bytes_to_bits(plaintext1)
                pt2_bits = self._bytes_to_bits(plaintext2)
                ct1_bits = self._bytes_to_bits(ciphertext1)
                ct2_bits = self._bytes_to_bits(ciphertext2)
                
                # XOR the ciphertexts to get the output difference
                output_diff = np.bitwise_xor(ct1_bits, ct2_bits)
                
                # Store plaintext pair and output difference
                X_batch[i, :64] = pt1_bits
                X_batch[i, 64:] = pt2_bits
                y_batch[i] = output_diff
            
            X_list.append(X_batch)
            y_list.append(y_batch)
            
            # Print progress
            if (batch + 1) % 5 == 0:
                print(f"Generated {(batch + 1) * batch_size} samples out of {num_samples}")
        
        # Concatenate all batches
        X = np.concatenate(X_list, axis=0)
        y = np.concatenate(y_list, axis=0)
        
        return X, y
    
    def _bytes_to_bits(self, byte_array):
        """Convert bytes to a numpy array of bits"""
        result = np.zeros(len(byte_array) * 8, dtype=np.float32)
        for i, byte in enumerate(byte_array):
            for j in range(8):
                result[i * 8 + j] = (byte >> (7 - j)) & 1
        return result
    
    def _bits_to_bytes(self, bit_array):
        """Convert a numpy array of bits to bytes"""
        # Convert float bits to uint8
        bit_array = bit_array.astype(np.uint8)
        result = bytearray(len(bit_array) // 8)
        for i in range(len(bit_array) // 8):
            byte = 0
            for j in range(8):
                byte |= bit_array[i * 8 + j] << (7 - j)
            result[i] = byte
        return bytes(result)
    
    def build_model(self):
        """Build and compile the neural network model with optimizations for M2"""
        # Clear any existing models to free memory
        tf.keras.backend.clear_session()
        gc.collect()
        
        # Use a smaller, more efficient model architecture
        inputs = Input(shape=(128,))
        
        # First dense layer with fewer neurons
        x = Dense(128, activation='relu')(inputs)
        x = BatchNormalization()(x)
        
        # Second dense layer
        x = Dense(256, activation='relu')(x)
        x = BatchNormalization()(x)
        
        # Final layer
        outputs = Dense(64, activation='sigmoid')(x)
        
        model = Model(inputs=inputs, outputs=outputs)
        
        # Use mixed precision for faster computation on M2
        # This uses the Neural Engine more efficiently
        if tf.config.list_physical_devices('GPU'):
            policy = tf.keras.mixed_precision.Policy('mixed_float16')
            tf.keras.mixed_precision.set_global_policy(policy)
        
        # Compile with efficient optimizer settings
        model.compile(
            optimizer=Adam(learning_rate=0.001),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        self.model = model
        return model
    
    def train_model(self, X, y, epochs=15, validation_split=0.2):
        """Train the model on the generated dataset"""
        if self.model is None:
            self.build_model()
            
        # Split into training and validation sets
        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=validation_split, random_state=42
        )
        
        # Free up memory
        del X, y
        gc.collect()
        
        # Callbacks for efficient training
        callbacks = [
            EarlyStopping(
                monitor='val_loss',
                patience=3,
                restore_best_weights=True
            ),
            ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=2,
                min_lr=0.0001
            )
        ]
        
        # Train the model with optimized batch size
        history = self.model.fit(
            X_train, y_train,
            epochs=epochs,
            batch_size=self.batch_size,
            validation_data=(X_val, y_val),
            callbacks=callbacks,
            verbose=1
        )
        
        # Free up memory after training
        del X_train, y_train
        gc.collect()
        
        return history
    
    def evaluate(self, X_test, y_test):
        """Evaluate the model on test data"""
        # Evaluate in batches to avoid memory issues
        loss, accuracy = self.model.evaluate(
            X_test, y_test, 
            batch_size=self.batch_size,
            verbose=1
        )
        print(f"Test Loss: {loss:.4f}")
        print(f"Test Accuracy: {accuracy:.4f}")
        return loss, accuracy
    
    def predict_key_bits(self, num_samples=500, target_round_key_idx=-1):
        """
        Try to predict bits of a target round key using differential patterns
        
        Args:
            num_samples: Number of plaintext pairs to analyze
            target_round_key_idx: Index of the round key to analyze (last round is most vulnerable)
            
        Returns:
            predicted_key_bits: Predicted bits of the target round key
        """
        # Generate new key and data
        key_bytes = 16  # Using 128-bit key
        secret_key = bytes([random.randint(0, 255) for _ in range(key_bytes)])
        cipher = Present(secret_key, rounds=self.rounds)
        
        # Get actual round keys for comparison
        actual_round_keys = cipher.roundkeys
        target_round_key = actual_round_keys[target_round_key_idx]
        
        # Prepare data structure to collect statistics
        key_bit_correlations = np.zeros(64)
        
        # Process in smaller batches
        batch_size = 100
        num_batches = num_samples // batch_size
        
        for batch in range(num_batches):
            X_pred_batch = np.zeros((batch_size, 128), dtype=np.float32)
            actual_diffs = np.zeros((batch_size, 64), dtype=np.float32)
            
            for i in range(batch_size):
                # Generate plaintext pair with specific difference
                plaintext1 = bytes([random.randint(0, 255) for _ in range(8)])
                plaintext_bytes = bytearray(plaintext1)
                plaintext_bytes[0] ^= 0x01
                plaintext2 = bytes(plaintext_bytes)
                
                # Encrypt both plaintexts
                ciphertext1 = cipher.encrypt(plaintext1)
                ciphertext2 = cipher.encrypt(plaintext2)
                
                # Get output difference
                ct1_bits = self._bytes_to_bits(ciphertext1)
                ct2_bits = self._bytes_to_bits(ciphertext2)
                output_diff = np.bitwise_xor(ct1_bits, ct2_bits)
                
                # Prepare input for prediction
                pt1_bits = self._bytes_to_bits(plaintext1)
                pt2_bits = self._bytes_to_bits(plaintext2)
                X_pred_batch[i, :64] = pt1_bits
                X_pred_batch[i, 64:] = pt2_bits
                actual_diffs[i] = output_diff
            
            # Make predictions for the batch
            predicted_diffs = (self.model.predict(X_pred_batch, batch_size=self.batch_size) > 0.5).astype(np.float32)
            
            # Compare predicted and actual differences
            matches = (predicted_diffs == actual_diffs).astype(np.float32)
            
            # Sum the matches for this batch
            key_bit_correlations += np.sum(matches, axis=0)
            
            # Print progress
            if (batch + 1) % 2 == 0:
                print(f"Processed {(batch + 1) * batch_size} samples out of {num_samples}")
        
        # Normalize correlations
        key_bit_correlations /= num_samples
        
        # Bits with high correlation are more likely to be influenced by the specific key bits
        threshold = 0.7
        predicted_key_bits = (key_bit_correlations > threshold).astype(np.uint8)
        
        # Display results
        print("Differential cryptanalysis results:")
        print(f"Actual round key #{target_round_key_idx}: {target_round_key:016x}")
        
        # Convert predicted bits to bytes for comparison (simplified)
        predicted_bytes = self._bits_to_bytes(predicted_key_bits)
        predicted_key_int = int.from_bytes(predicted_bytes, byteorder='big')
        print(f"Predicted key bits: {predicted_key_int:016x}")
        
        return predicted_key_bits, key_bit_correlations
    
    def plot_key_correlations(self, correlations):
        """Plot the key bit correlations"""
        plt.figure(figsize=(10, 5))
        plt.bar(range(64), correlations)
        plt.axhline(y=0.7, color='r', linestyle='-')
        plt.xlabel('Bit Position')
        plt.ylabel('Correlation')
        plt.title('Key Bit Correlation in Differential Analysis')
        plt.savefig('key_correlations.png')
        plt.close()
        
        print("Correlation plot saved as 'key_correlations.png'")

# Main execution for demonstration
def main():
    # Reduce model complexity for MacBook M2
    print("Starting differential cryptanalysis with optimized settings for M2 MacBook...")
    
    # Create the model with fewer rounds to reduce complexity
    dc_model = DifferentialCryptanalysisModel(rounds=3)
    
    # Generate smaller dataset
    print("Generating dataset...")
    X, y = dc_model.generate_dataset(num_samples=5000)
    
    # Build and train the model
    print("Building model optimized for M2...")
    dc_model.build_model()
    
    print("Training model...")
    history = dc_model.train_model(X, y, epochs=15)
    
    # Save model to file
    dc_model.model.save('present_diff_model_m2.h5')
    print("Model saved to 'present_diff_model_m2.h5'")
    
    # Plot training history and save to file
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper right')
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='lower right')
    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.close()
    
    print("Training history saved as 'training_history.png'")
    
    # Generate test data
    print("Generating test data...")
    X_test, y_test = dc_model.generate_dataset(num_samples=1000)
    
    # Evaluate model
    print("Evaluating model...")
    dc_model.evaluate(X_test, y_test)
    
    # Free memory
    del X_test, y_test
    gc.collect()
    
    # Attempt key recovery with fewer samples
    print("Performing key bit analysis...")
    predicted_bits, correlations = dc_model.predict_key_bits(num_samples=500)
    
    # Plot correlations
    dc_model.plot_key_correlations(correlations)
    
    print("Analysis complete!")

if __name__ == "__main__":
    main()

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import random
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import gc
import os

# Enable memory growth to prevent TensorFlow from allocating all GPU memory at once
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)

# Import the PRESENT cipher implementation
# Assumes the pypresent.py file is in the same directory
from pypresent import Present

class DifferentialCryptanalysisModel:
    def __init__(self, rounds=3):
        """
        Initialize the differential cryptanalysis model for PRESENT cipher
        
        Args:
            rounds: Number of rounds to analyze (reduced round version)
        """
        self.rounds = rounds
        self.present = None
        self.model = None
        self.batch_size = 256  # Increased batch size for better performance
        
    def generate_dataset(self, num_samples=2000, key_bits=128):
        """
        Generate dataset of plaintext-ciphertext pairs for training
        
        Args:
            num_samples: Number of samples to generate (reduced for faster execution)
            key_bits: Key size in bits (80 or 128)
        
        Returns:
            X: Input features (contains plaintext pairs with specific differences)
            y: Target values (output differences in specific bits)
        """
        # Generate a random key
        if key_bits == 80:
            key_bytes = 10
        else:
            key_bytes = 16
            
        key = bytes([random.randint(0, 255) for _ in range(key_bytes)])
        self.present = Present(key, rounds=self.rounds)
        
        # Generate data in smaller batches to avoid memory issues
        batch_size = 500  # Increased for faster processing
        num_batches = num_samples // batch_size
        
        X_list = []
        y_list = []
        
        for batch in range(num_batches):
            # Arrays to store our batch data
            X_batch = np.zeros((batch_size, 128), dtype=np.float32)
            y_batch = np.zeros((batch_size, 64), dtype=np.float32)
            
            # Pre-generate all plaintext pairs for this batch
            plaintexts1 = [bytes([random.randint(0, 255) for _ in range(8)]) for _ in range(batch_size)]
            plaintexts2 = []
            for pt1 in plaintexts1:
                pt2_bytes = bytearray(pt1)
                pt2_bytes[0] ^= 0x01  # Introducing a difference in the MSB of first byte
                plaintexts2.append(bytes(pt2_bytes))
            
            # Encrypt all plaintexts in batch (could be parallelized further)
            ciphertexts1 = [self.present.encrypt(pt) for pt in plaintexts1]
            ciphertexts2 = [self.present.encrypt(pt) for pt in plaintexts2]
            
            # Process all pairs
            for i in range(batch_size):
                # Convert to bit arrays
                pt1_bits = self._bytes_to_bits(plaintexts1[i])
                pt2_bits = self._bytes_to_bits(plaintexts2[i])
                ct1_bits = self._bytes_to_bits(ciphertexts1[i])
                ct2_bits = self._bytes_to_bits(ciphertexts2[i])
                
                # XOR the ciphertexts to get the output difference
                output_diff = np.bitwise_xor(ct1_bits, ct2_bits)
                
                # Store plaintext pair and output difference
                X_batch[i, :64] = pt1_bits
                X_batch[i, 64:] = pt2_bits
                y_batch[i] = output_diff
            
            X_list.append(X_batch)
            y_list.append(y_batch)
            
            # Print progress less frequently
            if (batch + 1) % 2 == 0:
                print(f"Generated {(batch + 1) * batch_size} samples out of {num_samples}")
        
        # Concatenate all batches
        X = np.concatenate(X_list, axis=0)
        y = np.concatenate(y_list, axis=0)
        
        return X, y
    
    def _bytes_to_bits(self, byte_array):
        """Convert bytes to a numpy array of bits - optimized version"""
        result = np.unpackbits(np.frombuffer(byte_array, dtype=np.uint8))
        return result.astype(np.float32)
    
    def _bits_to_bytes(self, bit_array):
        """Convert a numpy array of bits to bytes - optimized version"""
        bit_array = bit_array.astype(np.uint8)
        result = np.packbits(bit_array)
        return bytes(result)
    
    def build_model(self):
        """Build and compile a simpler neural network model"""
        # Clear any existing models to free memory
        tf.keras.backend.clear_session()
        gc.collect()
        
        # Use a smaller, more efficient model architecture
        inputs = Input(shape=(128,))
        
        # Simpler architecture with fewer parameters
        x = Dense(64, activation='relu')(inputs)
        x = BatchNormalization()(x)
        x = Dropout(0.2)(x)  # Added dropout to prevent overfitting and speed up training
        
        x = Dense(128, activation='relu')(x)
        x = BatchNormalization()(x)
        
        # Final layer
        outputs = Dense(64, activation='sigmoid')(x)
        
        model = Model(inputs=inputs, outputs=outputs)
        
        # Use mixed precision for faster computation
        if tf.config.list_physical_devices('GPU'):
            policy = tf.keras.mixed_precision.Policy('mixed_float16')
            tf.keras.mixed_precision.set_global_policy(policy)
        
        # Compile with efficient optimizer settings
        model.compile(
            optimizer=Adam(learning_rate=0.002),  # Slightly higher learning rate
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        self.model = model
        return model
    
    def train_model(self, X, y, epochs=10, validation_split=0.2):  # Reduced epochs
        """Train the model on the generated dataset with early stopping"""
        if self.model is None:
            self.build_model()
            
        # Split into training and validation sets
        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=validation_split, random_state=42
        )
        
        # Free up memory
        del X, y
        gc.collect()
        
        # More aggressive early stopping to prevent unnecessary training
        callbacks = [
            EarlyStopping(
                monitor='val_loss',
                patience=2,  # Reduced patience
                restore_best_weights=True
            ),
            ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=1,  # Reduced patience
                min_lr=0.0001
            )
        ]
        
        # Train the model with optimized batch size
        history = self.model.fit(
            X_train, y_train,
            epochs=epochs,
            batch_size=self.batch_size,
            validation_data=(X_val, y_val),
            callbacks=callbacks,
            verbose=1
        )
        
        # Free up memory after training
        del X_train, y_train
        gc.collect()
        
        return history
    
    def evaluate(self, X_test, y_test):
        """Evaluate the model on test data"""
        # Evaluate in batches to avoid memory issues
        loss, accuracy = self.model.evaluate(
            X_test, y_test, 
            batch_size=self.batch_size,
            verbose=1
        )
        print(f"Test Loss: {loss:.4f}")
        print(f"Test Accuracy: {accuracy:.4f}")
        return loss, accuracy
    
    def predict_key_bits(self, num_samples=200, target_round_key_idx=-1):  # Reduced samples
        """
        Try to predict bits of a target round key using differential patterns
        
        Args:
            num_samples: Number of plaintext pairs to analyze (reduced)
            target_round_key_idx: Index of the round key to analyze
            
        Returns:
            predicted_key_bits: Predicted bits of the target round key
        """
        # Generate new key and data
        key_bytes = 16  # Using 128-bit key
        secret_key = bytes([random.randint(0, 255) for _ in range(key_bytes)])
        cipher = Present(secret_key, rounds=self.rounds)
        
        # Get actual round keys for comparison
        actual_round_keys = cipher.roundkeys
        target_round_key = actual_round_keys[target_round_key_idx]
        
        # Prepare data structure to collect statistics
        key_bit_correlations = np.zeros(64)
        
        # Process in larger batches
        batch_size = 100
        num_batches = num_samples // batch_size
        
        for batch in range(num_batches):
            X_pred_batch = np.zeros((batch_size, 128), dtype=np.float32)
            actual_diffs = np.zeros((batch_size, 64), dtype=np.float32)
            
            # Pre-generate all plaintexts
            plaintexts1 = [bytes([random.randint(0, 255) for _ in range(8)]) for _ in range(batch_size)]
            plaintexts2 = []
            for pt1 in plaintexts1:
                pt2_bytes = bytearray(pt1)
                pt2_bytes[0] ^= 0x01
                plaintexts2.append(bytes(pt2_bytes))
            
            # Encrypt all plaintexts
            ciphertexts1 = [cipher.encrypt(pt) for pt in plaintexts1]
            ciphertexts2 = [cipher.encrypt(pt) for pt in plaintexts2]
            
            for i in range(batch_size):
                # Get output difference
                ct1_bits = self._bytes_to_bits(ciphertexts1[i])
                ct2_bits = self._bytes_to_bits(ciphertexts2[i])
                output_diff = np.bitwise_xor(ct1_bits, ct2_bits)
                
                # Prepare input for prediction
                pt1_bits = self._bytes_to_bits(plaintexts1[i])
                pt2_bits = self._bytes_to_bits(plaintexts2[i])
                X_pred_batch[i, :64] = pt1_bits
                X_pred_batch[i, 64:] = pt2_bits
                actual_diffs[i] = output_diff
            
            # Make predictions for the batch
            predicted_diffs = (self.model.predict(X_pred_batch, batch_size=self.batch_size, verbose=0) > 0.5).astype(np.float32)
            
            # Compare predicted and actual differences
            matches = (predicted_diffs == actual_diffs).astype(np.float32)
            
            # Sum the matches for this batch
            key_bit_correlations += np.sum(matches, axis=0)
        
        # Normalize correlations
        key_bit_correlations /= num_samples
        
        # Bits with high correlation are more likely to be influenced by the specific key bits
        threshold = 0.7
        predicted_key_bits = (key_bit_correlations > threshold).astype(np.uint8)
        
        # Display results
        print("Differential cryptanalysis results:")
        print(f"Actual round key #{target_round_key_idx}: {target_round_key:016x}")
        
        # Convert predicted bits to bytes for comparison
        predicted_bytes = self._bits_to_bytes(predicted_key_bits)
        predicted_key_int = int.from_bytes(predicted_bytes, byteorder='big')
        print(f"Predicted key bits: {predicted_key_int:016x}")
        
        return predicted_key_bits, key_bit_correlations
    
    def plot_key_correlations(self, correlations):
        """Plot the key bit correlations"""
        plt.figure(figsize=(10, 5))
        plt.bar(range(64), correlations)
        plt.axhline(y=0.7, color='r', linestyle='-')
        plt.xlabel('Bit Position')
        plt.ylabel('Correlation')
        plt.title('Key Bit Correlation in Differential Analysis')
        plt.savefig('key_correlations.png')
        plt.close()
        
        print("Correlation plot saved as 'key_correlations.png'")

# Main execution with optimized parameters
def main():
    print("Starting optimized differential cryptanalysis...")
    
    # Create the model with fewer rounds to reduce complexity
    dc_model = DifferentialCryptanalysisModel(rounds=3)
    
    # Generate smaller dataset
    print("Generating dataset (reduced size)...")
    X, y = dc_model.generate_dataset(num_samples=2000)  # Reduced from 5000
    
    # Build and train the model
    print("Building optimized model...")
    dc_model.build_model()
    
    print("Training model (faster convergence)...")
    history = dc_model.train_model(X, y, epochs=10)  # Reduced from 15
    
    # Save model to file
    dc_model.model.save('present_diff_model_optimized.h5')
    print("Model saved to 'present_diff_model_optimized.h5'")
    
    # Plot training history and save to file
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper right')
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='lower right')
    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.close()
    
    print("Training history saved as 'training_history.png'")
    
    # Generate test data (smaller set)
    print("Generating test data...")
    X_test, y_test = dc_model.generate_dataset(num_samples=500)  # Reduced from 1000
    
    # Evaluate model
    print("Evaluating model...")
    dc_model.evaluate(X_test, y_test)
    
    # Free memory
    del X_test, y_test
    gc.collect()
    
    # Attempt key recovery with fewer samples
    print("Performing key bit analysis...")
    predicted_bits, correlations = dc_model.predict_key_bits(num_samples=200)  # Reduced from 500
    
    # Plot correlations
    dc_model.plot_key_correlations(correlations)
    
    print("Analysis complete!")

if __name__ == "__main__":
    main()

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_metal  # This enables Metal GPU support on Mac
from sklearn.model_selection import train_test_split
import random
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import gc
import os
import multiprocessing
from concurrent.futures import ThreadPoolExecutor

# Configure TensorFlow to use Metal
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Enable Metal plugin for TensorFlow
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
    print("Metal GPU enabled for TensorFlow")
else:
    print("No Metal GPU found, running on CPU")

# Import the PRESENT cipher implementation
# Assumes the pypresent.py file is in the same directory
from pypresent import Present

class DifferentialCryptanalysisModel:
    def __init__(self, rounds=2):  # Reduced default rounds to 2 for faster execution
        """
        Initialize the differential cryptanalysis model for PRESENT cipher
        
        Args:
            rounds: Number of rounds to analyze (reduced round version)
        """
        self.rounds = rounds
        self.present = None
        self.model = None
        self.batch_size = 512  # Larger batch size for better GPU utilization
        self.num_threads = max(1, multiprocessing.cpu_count() - 1)  # Leave one CPU free
        
    def _encrypt_pair(self, pair_data):
        """Helper function for parallel encryption"""
        plaintext, cipher = pair_data
        return cipher.encrypt(plaintext)
        
    def generate_dataset(self, num_samples=1500, key_bits=128):
        """
        Generate dataset of plaintext-ciphertext pairs for training
        
        Args:
            num_samples: Number of samples to generate (reduced)
            key_bits: Key size in bits (80 or 128)
        
        Returns:
            X: Input features (contains plaintext pairs with specific differences)
            y: Target values (output differences in specific bits)
        """
        # Generate a random key
        if key_bits == 80:
            key_bytes = 10
        else:
            key_bytes = 16
            
        key = bytes([random.randint(0, 255) for _ in range(key_bytes)])
        self.present = Present(key, rounds=self.rounds)
        
        # Generate data in batches for better memory management
        batch_size = 500
        num_batches = num_samples // batch_size
        
        X_list = []
        y_list = []
        
        for batch in range(num_batches):
            # Arrays to store our batch data
            X_batch = np.zeros((batch_size, 128), dtype=np.float32)
            y_batch = np.zeros((batch_size, 64), dtype=np.float32)
            
            # Pre-generate all plaintext pairs for this batch
            plaintexts1 = [bytes([random.randint(0, 255) for _ in range(8)]) for _ in range(batch_size)]
            plaintexts2 = []
            for pt1 in plaintexts1:
                pt2_bytes = bytearray(pt1)
                pt2_bytes[0] ^= 0x01  # Introducing a difference in the MSB of first byte
                plaintexts2.append(bytes(pt2_bytes))
            
            # Parallelize encryption using ThreadPoolExecutor
            with ThreadPoolExecutor(max_workers=self.num_threads) as executor:
                # Encrypt plaintexts1
                cipher_pairs1 = [(pt, self.present) for pt in plaintexts1]
                ciphertexts1 = list(executor.map(self._encrypt_pair, cipher_pairs1))
                
                # Encrypt plaintexts2
                cipher_pairs2 = [(pt, self.present) for pt in plaintexts2]
                ciphertexts2 = list(executor.map(self._encrypt_pair, cipher_pairs2))
            
            # Process all pairs
            for i in range(batch_size):
                # Convert to bit arrays
                pt1_bits = self._bytes_to_bits(plaintexts1[i])
                pt2_bits = self._bytes_to_bits(plaintexts2[i])
                ct1_bits = self._bytes_to_bits(ciphertexts1[i])
                ct2_bits = self._bytes_to_bits(ciphertexts2[i])
                
                # XOR the ciphertexts to get the output difference
                output_diff = np.bitwise_xor(ct1_bits, ct2_bits)
                
                # Store plaintext pair and output difference
                X_batch[i, :64] = pt1_bits
                X_batch[i, 64:] = pt2_bits
                y_batch[i] = output_diff
            
            X_list.append(X_batch)
            y_list.append(y_batch)
            
            # Print progress
            print(f"Generated {(batch + 1) * batch_size} samples out of {num_samples}")
        
        # Concatenate all batches
        X = np.concatenate(X_list, axis=0)
        y = np.concatenate(y_list, axis=0)
        
        return X, y
    
    def _bytes_to_bits(self, byte_array):
        """Convert bytes to a numpy array of bits - vectorized version"""
        return np.unpackbits(np.frombuffer(byte_array, dtype=np.uint8)).astype(np.float32)
    
    def _bits_to_bytes(self, bit_array):
        """Convert a numpy array of bits to bytes - vectorized version"""
        return bytes(np.packbits(bit_array.astype(np.uint8)))
    
    def build_model(self):
        """Build and compile a Mac Metal-optimized neural network model"""
        # Clear any existing models to free memory
        tf.keras.backend.clear_session()
        gc.collect()
        
        # Metal-optimized model architecture
        inputs = Input(shape=(128,))
        
        # First layer - keep smaller for faster processing
        x = Dense(64, activation='relu')(inputs)
        x = BatchNormalization()(x)
        x = Dropout(0.2)(x)
        
        # Optional middle layer - comment out for a faster but slightly less accurate model
        # x = Dense(64, activation='relu')(x)
        # x = BatchNormalization()(x)
        # x = Dropout(0.2)(x)
        
        # Output layer
        outputs = Dense(64, activation='sigmoid')(x)
        
        model = Model(inputs=inputs, outputs=outputs)
        
        # Enable mixed precision for Metal GPU acceleration
        if tf.config.list_physical_devices('GPU'):
            try:
                policy = tf.keras.mixed_precision.Policy('mixed_float16')
                tf.keras.mixed_precision.set_global_policy(policy)
                print("Using mixed precision for faster GPU computation")
            except:
                print("Mixed precision not supported, using default precision")
        
        # Compile with optimized settings for Metal
        model.compile(
            optimizer=Adam(learning_rate=0.002),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        # Print model summary
        model.summary()
        
        self.model = model
        return model
    
    def train_model(self, X, y, epochs=8, validation_split=0.2):
        """Train the model with Metal GPU optimization"""
        if self.model is None:
            self.build_model()
            
        # Split into training and validation sets
        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=validation_split, random_state=42
        )
        
        # Free up memory
        del X, y
        gc.collect()
        
        # Callbacks optimized for faster convergence
        callbacks = [
            EarlyStopping(
                monitor='val_loss',
                patience=2,
                restore_best_weights=True,
                verbose=1
            ),
            ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=1,
                min_lr=0.0001,
                verbose=1
            )
        ]
        
        # Train with optimized settings for Metal
        history = self.model.fit(
            X_train, y_train,
            epochs=epochs,
            batch_size=self.batch_size,
            validation_data=(X_val, y_val),
            callbacks=callbacks,
            verbose=1,
            # Add the following options for Metal optimization
            use_multiprocessing=True,
            workers=self.num_threads
        )
        
        # Free up memory
        del X_train, y_train
        gc.collect()
        
        return history
    
    def evaluate(self, X_test, y_test):
        """Evaluate the model on test data"""
        # Evaluate in batches with Metal GPU optimization
        loss, accuracy = self.model.evaluate(
            X_test, y_test, 
            batch_size=self.batch_size,
            verbose=1,
            use_multiprocessing=True,
            workers=self.num_threads
        )
        print(f"Test Loss: {loss:.4f}")
        print(f"Test Accuracy: {accuracy:.4f}")
        return loss, accuracy
    
    def predict_key_bits(self, num_samples=150, target_round_key_idx=-1):
        """
        Try to predict bits of a target round key using differential patterns
        (Optimized for Mac performance)
        """
        # Generate new key and data
        key_bytes = 16  # Using 128-bit key
        secret_key = bytes([random.randint(0, 255) for _ in range(key_bytes)])
        cipher = Present(secret_key, rounds=self.rounds)
        
        # Get actual round keys for comparison
        actual_round_keys = cipher.roundkeys
        target_round_key = actual_round_keys[target_round_key_idx]
        
        # Prepare data structure to collect statistics
        key_bit_correlations = np.zeros(64)
        
        # Process in larger batches for better GPU utilization
        batch_size = 50
        num_batches = num_samples // batch_size
        
        for batch in range(num_batches):
            X_pred_batch = np.zeros((batch_size, 128), dtype=np.float32)
            actual_diffs = np.zeros((batch_size, 64), dtype=np.float32)
            
            # Pre-generate all plaintexts
            plaintexts1 = [bytes([random.randint(0, 255) for _ in range(8)]) for _ in range(batch_size)]
            plaintexts2 = []
            for pt1 in plaintexts1:
                pt2_bytes = bytearray(pt1)
                pt2_bytes[0] ^= 0x01
                plaintexts2.append(bytes(pt2_bytes))
            
            # Parallelize encryption
            with ThreadPoolExecutor(max_workers=self.num_threads) as executor:
                # Encrypt plaintexts1
                cipher_pairs1 = [(pt, cipher) for pt in plaintexts1]
                ciphertexts1 = list(executor.map(self._encrypt_pair, cipher_pairs1))
                
                # Encrypt plaintexts2
                cipher_pairs2 = [(pt, cipher) for pt in plaintexts2]
                ciphertexts2 = list(executor.map(self._encrypt_pair, cipher_pairs2))
            
            for i in range(batch_size):
                # Get output difference
                ct1_bits = self._bytes_to_bits(ciphertexts1[i])
                ct2_bits = self._bytes_to_bits(ciphertexts2[i])
                output_diff = np.bitwise_xor(ct1_bits, ct2_bits)
                
                # Prepare input for prediction
                pt1_bits = self._bytes_to_bits(plaintexts1[i])
                pt2_bits = self._bytes_to_bits(plaintexts2[i])
                X_pred_batch[i, :64] = pt1_bits
                X_pred_batch[i, 64:] = pt2_bits
                actual_diffs[i] = output_diff
            
            # Make predictions for the batch
            predicted_diffs = (self.model.predict(
                X_pred_batch, 
                batch_size=self.batch_size, 
                verbose=0,
                use_multiprocessing=True,
                workers=self.num_threads
            ) > 0.5).astype(np.float32)
            
            # Compare predicted and actual differences
            matches = (predicted_diffs == actual_diffs).astype(np.float32)
            
            # Sum the matches for this batch
            key_bit_correlations += np.sum(matches, axis=0)
            
            print(f"Processed {(batch + 1) * batch_size} samples out of {num_samples}")
        
        # Normalize correlations
        key_bit_correlations /= num_samples
        
        # Bits with high correlation are more likely to be influenced by the specific key bits
        threshold = 0.7
        predicted_key_bits = (key_bit_correlations > threshold).astype(np.uint8)
        
        # Display results
        print("Differential cryptanalysis results:")
        print(f"Actual round key #{target_round_key_idx}: {target_round_key:016x}")
        
        # Convert predicted bits to bytes for comparison
        predicted_bytes = self._bits_to_bytes(predicted_key_bits)
        predicted_key_int = int.from_bytes(predicted_bytes, byteorder='big')
        print(f"Predicted key bits: {predicted_key_int:016x}")
        
        return predicted_key_bits, key_bit_correlations
    
    def plot_key_correlations(self, correlations):
        """Plot the key bit correlations"""
        plt.figure(figsize=(10, 5))
        plt.bar(range(64), correlations)
        plt.axhline(y=0.7, color='r', linestyle='-')
        plt.xlabel('Bit Position')
        plt.ylabel('Correlation')
        plt.title('Key Bit Correlation in Differential Analysis')
        plt.savefig('key_correlations.png')
        plt.close()
        
        print("Correlation plot saved as 'key_correlations.png'")

# Main execution with Mac-optimized parameters
def main():
    print("Starting Mac Metal GPU optimized differential cryptanalysis...")
    
    # Create the model with reduced rounds
    dc_model = DifferentialCryptanalysisModel(rounds=2)  # Reduced to 2 rounds
    
    # Generate smaller dataset
    print("Generating dataset (optimized for Mac)...")
    X, y = dc_model.generate_dataset(num_samples=1500)  # Further reduced
    
    # Build and train the model
    print("Building model with Metal GPU optimization...")
    dc_model.build_model()
    
    print("Training model (Metal-optimized)...")
    history = dc_model.train_model(X, y, epochs=8)  # Fewer epochs
    
    # Save model to file
    dc_model.model.save('present_diff_model_mac.h5')
    print("Model saved to 'present_diff_model_mac.h5'")
    
    # Plot training history
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper right')
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='lower right')
    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.close()
    
    # Generate test data (smaller set)
    print("Generating test data...")
    X_test, y_test = dc_model.generate_dataset(num_samples=300)  # Smaller test set
    
    # Evaluate model
    print("Evaluating model...")
    dc_model.evaluate(X_test, y_test)
    
    # Free memory
    del X_test, y_test
    gc.collect()
    
    # Attempt key recovery with fewer samples
    print("Performing key bit analysis...")
    predicted_bits, correlations = dc_model.predict_key_bits(num_samples=150)  # Further reduced
    
    # Plot correlations
    dc_model.plot_key_correlations(correlations)
    
    print("Mac-optimized analysis complete!")

if __name__ == "__main__":
    main()

In [ ]:
import tensorflow as tf
import numpy as np
from pypresent import Present
import random

class PresentDifferentialAttack:
    def __init__(self, num_rounds=5):
        """Initialize the attack on a reduced-round PRESENT cipher
        
        Args:
            num_rounds: Number of rounds to attack (full cipher uses 32)
        """
        self.num_rounds = num_rounds
        self.block_size = 64  # PRESENT uses 64-bit blocks
        self.model = None
        
    def generate_training_data(self, num_samples=10000):
        """Generate training data from input/output differences
        
        Returns:
            X: Input differences
            y: Last round subkey candidates (one-hot encoded)
        """
        # Generate random keys for training
        keys = [bytes([random.randint(0, 255) for _ in range(16)]) for _ in range(20)]
        
        X = []
        y = []
        
        for _ in range(num_samples):
            # Choose a random key for this sample
            key = random.choice(keys)
            cipher = Present(key, rounds=self.num_rounds)
            
            # Generate a plaintext and a differential
            plaintext1 = bytes([random.randint(0, 255) for _ in range(8)])
            
            # Create a plaintext with a specific difference
            diff_position = random.randint(0, 63)
            diff_mask = 1 << diff_position
            
            # Convert to integer for easier bit manipulation
            pt1_int = int.from_bytes(plaintext1, byteorder='big')
            pt2_int = pt1_int ^ diff_mask
            plaintext2 = pt2_int.to_bytes(8, byteorder='big')
            
            # Encrypt both plaintexts
            ciphertext1 = cipher.encrypt(plaintext1)
            ciphertext2 = cipher.encrypt(plaintext2)
            
            # Calculate input and output differences
            input_diff = pt1_int ^ pt2_int
            output_diff = int.from_bytes(ciphertext1, byteorder='big') ^ int.from_bytes(ciphertext2, byteorder='big')
            
            # Convert to binary feature vectors
            input_diff_binary = self._int_to_binary(input_diff, self.block_size)
            output_diff_binary = self._int_to_binary(output_diff, self.block_size)
            
            # For simplicity, target is the last round key (simplified)
            # In a real attack, would be probabilities or information about key bits
            last_round_key = cipher.roundkeys[-1] & 0xFFFFFFFF  # Take lower 32 bits for demonstration
            target = self._int_to_binary(last_round_key, 32)
            
            X.append(np.concatenate([input_diff_binary, output_diff_binary]))
            y.append(target)
        
        return np.array(X), np.array(y)
    
    def _int_to_binary(self, n, bits):
        """Convert integer to binary array"""
        return np.array([(n >> i) & 1 for i in range(bits)])
    
    def build_model(self):
        """Build a neural network for the differential analysis"""
        input_size = self.block_size * 2  # Input diff and output diff
        output_size = 32  # Predicting bits of the last round key (simplified)
        
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(512, activation='relu', input_shape=(input_size,)),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(output_size, activation='sigmoid')
        ])
        
        model.compile(
            optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        self.model = model
        return model
    
    def train(self, epochs=50, batch_size=128):
        """Train the model on differential pairs"""
        if self.model is None:
            self.build_model()
            
        X, y = self.generate_training_data()
        
        # Split into training and validation sets
        split = int(0.8 * len(X))
        X_train, X_val = X[:split], X[split:]
        y_train, y_val = y[:split], y[split:]
        
        # Train the model
        history = self.model.fit(
            X_train, y_train,
            epochs=epochs,
            batch_size=batch_size,
            validation_data=(X_val, y_val),
            verbose=1
        )
        
        return history
    
    def attack(self, num_test_pairs=1000, actual_key=None):
        """Attempt to recover key bits from the trained model
        
        Args:
            num_test_pairs: Number of differential pairs to use in the attack
            actual_key: The true key for validation (if known)
            
        Returns:
            predicted_key_bits: The predicted key bits
        """
        if self.model is None:
            raise ValueError("Model must be trained before attacking")
            
        if actual_key:
            # For testing - create a cipher with the known key
            cipher = Present(actual_key, rounds=self.num_rounds)
            actual_last_round_key = cipher.roundkeys[-1] & 0xFFFFFFFF
            print(f"Actual last round key (32 bits): {actual_last_round_key:08x}")
        
        # Generate test differential pairs
        test_diffs = []
        for _ in range(num_test_pairs):
            # Similar to training data generation
            plaintext1 = bytes([random.randint(0, 255) for _ in range(8)])
            
            diff_position = random.randint(0, 63)
            diff_mask = 1 << diff_position
            
            pt1_int = int.from_bytes(plaintext1, byteorder='big')
            pt2_int = pt1_int ^ diff_mask
            plaintext2 = pt2_int.to_bytes(8, byteorder='big')
            
            # Encrypt with the target cipher
            ciphertext1 = cipher.encrypt(plaintext1)
            ciphertext2 = cipher.encrypt(plaintext2)
            
            # Calculate differences
            input_diff = pt1_int ^ pt2_int
            output_diff = int.from_bytes(ciphertext1, byteorder='big') ^ int.from_bytes(ciphertext2, byteorder='big')
            
            # Convert to binary
            input_diff_binary = self._int_to_binary(input_diff, self.block_size)
            output_diff_binary = self._int_to_binary(output_diff, self.block_size)
            
            test_diffs.append(np.concatenate([input_diff_binary, output_diff_binary]))
        
        # Predict key bits
        predictions = self.model.predict(np.array(test_diffs))
        
        # Average predictions over all test pairs
        avg_prediction = np.mean(predictions, axis=0)
        
        # Convert probabilities to binary (0 or 1)
        predicted_key_bits = (avg_prediction > 0.5).astype(int)
        
        # Convert binary array back to integer
        predicted_key = sum(bit << i for i, bit in enumerate(predicted_key_bits))
        
        print(f"Predicted last round key (32 bits): {predicted_key:08x}")
        
        if actual_key:
            # Calculate success rate
            correct_bits = sum(1 for a, b in zip(self._int_to_binary(actual_last_round_key, 32), predicted_key_bits) if a == b)
            success_rate = correct_bits / 32 * 100
            print(f"Success rate: {success_rate:.2f}% ({correct_bits}/32 bits correct)")
        
        return predicted_key

# Example usage
def demonstrate_attack():
    # Generate a random key for demonstration
    print("check 1")
    
    import os
    key = os.urandom(16)  # 128-bit key
    
    print("check 2")
    
    # Create the attack object (reduced to 5 rounds for demonstration)
    attack = PresentDifferentialAttack(num_rounds=5)
    
    print("check 3")
    
    # Build and train the model
    attack.build_model()
    history = attack.train(epochs=20)  # Reduced epochs for demonstration
    
    print("check 4")
    
    # Perform the attack
    recovered_key = attack.attack(actual_key=key)
    
    print("check 5")
    
    return recovered_key, history

if __name__ == "__main__":
    recovered_key, history = demonstrate_attack()

In [ ]:
import numpy as np
import random
import os
import sys
from tqdm import tqdm

# Inline implementation of the PRESENT cipher
class Present:
    def __init__(self, key, rounds=32):
        """Create a PRESENT cipher object

        key:    the key as a 128-bit or 80-bit bytes object
        rounds: the number of rounds as an integer, 32 by default
        """
        self.rounds = rounds
        if len(key) * 8 == 80:
            self.roundkeys = generateRoundkeys80(string2number(key), self.rounds)
        elif len(key) * 8 == 128:
            self.roundkeys = generateRoundkeys128(string2number(key), self.rounds)
        else:
            raise ValueError("Key must be a 128-bit or 80-bit bytes object")

    def encrypt(self, block):
        """Encrypt 1 block (8 bytes)

        Input:  plaintext block as bytes
        Output: ciphertext block as bytes
        """
        state = string2number(block)
        for i in range(self.rounds - 1):
            state = addRoundKey(state, self.roundkeys[i])
            state = sBoxLayer(state)
            state = pLayer(state)
        cipher = addRoundKey(state, self.roundkeys[-1])
        return number2string_N(cipher, 8)

    def decrypt(self, block):
        """Decrypt 1 block (8 bytes)

        Input:  ciphertext block as bytes
        Output: plaintext block as bytes
        """
        state = string2number(block)
        for i in range(self.rounds - 1):
            state = addRoundKey(state, self.roundkeys[-i - 1])
            state = pLayer_dec(state)
            state = sBoxLayer_dec(state)
        decipher = addRoundKey(state, self.roundkeys[0])
        return number2string_N(decipher, 8)

    def get_block_size(self):
        return 8

# 0   1   2   3   4   5   6   7   8   9   a   b   c   d   e   f
Sbox = [0xc, 0x5, 0x6, 0xb, 0x9, 0x0, 0xa, 0xd, 0x3, 0xe, 0xf, 0x8, 0x4, 0x7, 0x1, 0x2]
Sbox_inv = [Sbox.index(x) for x in range(16)]
PBox = [0, 16, 32, 48, 1, 17, 33, 49, 2, 18, 34, 50, 3, 19, 35, 51,
        4, 20, 36, 52, 5, 21, 37, 53, 6, 22, 38, 54, 7, 23, 39, 55,
        8, 24, 40, 56, 9, 25, 41, 57, 10, 26, 42, 58, 11, 27, 43, 59,
        12, 28, 44, 60, 13, 29, 45, 61, 14, 30, 46, 62, 15, 31, 47, 63]
PBox_inv = [PBox.index(x) for x in range(64)]

def generateRoundkeys80(key, rounds):
    """Generate the roundkeys for a 80-bit key

    Input:
            key:    the key as a 80-bit integer
            rounds: the number of rounds as an integer
    Output: list of 64-bit roundkeys as integers"""
    roundkeys = []
    for i in tqdm(range(1, rounds + 1), desc="Generating 80-bit roundkeys"):  # (K1 ... K32)
        # rawkey: used in comments to show what happens at bitlevel
        # rawKey[0:64]
        roundkeys.append(key >> 16)
        # 1. Shift
        # rawKey[19:len(rawKey)]+rawKey[0:19]
        key = ((key & (2 ** 19 - 1)) << 61) + (key >> 19)
        # 2. SBox
        # rawKey[76:80] = S(rawKey[76:80])
        key = (Sbox[key >> 76] << 76) + (key & (2 ** 76 - 1))
        #3. Salt
        #rawKey[15:20] ^ i
        key ^= i << 15
    return roundkeys

def generateRoundkeys128(key, rounds):
    """Generate the roundkeys for a 128-bit key

    Input:
            key:    the key as a 128-bit integer
            rounds: the number of rounds as an integer
    Output: list of 64-bit roundkeys as integers"""
    roundkeys = []
    for i in tqdm(range(1, rounds + 1), desc="Generating 128-bit roundkeys"):  # (K1 ... K32)
        # rawkey: used in comments to show what happens at bitlevel
        roundkeys.append(key >> 64)
        # 1. Shift
        key = ((key & (2 ** 67 - 1)) << 61) + (key >> 67)
        # 2. SBox
        key = (Sbox[key >> 124] << 124) + (Sbox[(key >> 120) & 0xF] << 120) + (key & (2 ** 120 - 1))
        # 3. Salt
        # rawKey[62:67] ^ i
        key ^= i << 62
    return roundkeys

def addRoundKey(state, roundkey):
    return state ^ roundkey

def sBoxLayer(state):
    """SBox function for encryption

    Input:  64-bit integer
    Output: 64-bit integer"""

    output = 0
    for i in range(16):
        output += Sbox[( state >> (i * 4)) & 0xF] << (i * 4)
    return output

def sBoxLayer_dec(state):
    """Inverse SBox function for decryption

    Input:  64-bit integer
    Output: 64-bit integer"""
    output = 0
    for i in range(16):
        output += Sbox_inv[( state >> (i * 4)) & 0xF] << (i * 4)
    return output

def pLayer(state):
    """Permutation layer for encryption

    Input:  64-bit integer
    Output: 64-bit integer"""
    output = 0
    for i in range(64):
        output += ((state >> i) & 0x01) << PBox[i]
    return output

def pLayer_dec(state):
    """Permutation layer for decryption

    Input:  64-bit integer
    Output: 64-bit integer"""
    output = 0
    for i in range(64):
        output += ((state >> i) & 0x01) << PBox_inv[i]
    return output

def string2number(i):
    """ Convert a bytes object to a number

    Input: bytes (big-endian)
    Output: integer
    """
    return int.from_bytes(i, byteorder='big')

def number2string_N(i, N):
    """Convert a number to a bytes object of fixed size

    i: integer
    N: length of bytes
    Output: bytes (big-endian)
    """
    return i.to_bytes(N, byteorder='big')


class PresentDifferentialAttack:
    def __init__(self, num_rounds=5):
        """Initialize the attack on a reduced-round PRESENT cipher
        
        Args:
            num_rounds: Number of rounds to attack (full cipher uses 32)
        """
        print("Initializing PresentDifferentialAttack with", num_rounds, "rounds")
        self.num_rounds = num_rounds
        self.block_size = 64  # PRESENT uses 64-bit blocks
        self.model = None
        
    def generate_training_data(self, num_samples=10000):
        """Generate training data from input/output differences
        
        Returns:
            X: Input differences
            y: Last round subkey candidates (one-hot encoded)
        """
        print(f"Generating {num_samples} training samples...")
        
        # Generate random keys for training (fewer keys to improve training signal)
        keys = [bytes([random.randint(0, 255) for _ in range(16)]) for _ in range(5)]
        
        X = []
        y = []
        
        for i in tqdm(range(num_samples), desc="Generating training samples"):
            try:
                # Choose a random key for this sample
                key = random.choice(keys)
                cipher = Present(key, rounds=self.num_rounds)
                
                # Generate a plaintext and a differential
                plaintext1 = bytes([random.randint(0, 255) for _ in range(8)])
                
                # Instead of random single-bit differences, use more structured differences
                # that might propagate better through the cipher
                if random.random() < 0.7:
                    # Single-bit difference (70% of the time)
                    diff_position = random.randint(0, 63)
                    diff_mask = 1 << diff_position
                else:
                    # Byte-level difference (30% of the time)
                    byte_pos = random.randint(0, 7)
                    byte_val = random.randint(1, 255)  # Non-zero difference
                    diff_mask = byte_val << (byte_pos * 8)
                
                # Convert to integer for easier bit manipulation
                pt1_int = int.from_bytes(plaintext1, byteorder='big')
                pt2_int = pt1_int ^ diff_mask
                plaintext2 = pt2_int.to_bytes(8, byteorder='big')
                
                # Encrypt both plaintexts
                ciphertext1 = cipher.encrypt(plaintext1)
                ciphertext2 = cipher.encrypt(plaintext2)
                
                # Calculate input and output differences
                input_diff = pt1_int ^ pt2_int
                output_diff = int.from_bytes(ciphertext1, byteorder='big') ^ int.from_bytes(ciphertext2, byteorder='big')
                
                # Convert to binary feature vectors
                input_diff_binary = self._int_to_binary(input_diff, self.block_size)
                output_diff_binary = self._int_to_binary(output_diff, self.block_size)
                
                # For training purposes, target is information about the last round key
                # We'll use the full 64 bits for better training signal
                last_round_key = cipher.roundkeys[-1]
                target = self._int_to_binary(last_round_key, 64)
                
                X.append(np.concatenate([input_diff_binary, output_diff_binary]))
                y.append(target)
                
            except Exception as e:
                print(f"Error generating sample: {e}")
                continue
        
        return np.array(X), np.array(y)

    def _int_to_binary(self, n, bits):
        """Convert integer to binary array"""
        return np.array([(n >> i) & 1 for i in range(bits)])

    def _binary_to_int(self, binary_array):
        """Convert binary array to integer"""
        return sum(bit << i for i, bit in enumerate(binary_array))
    
    def build_model(self):
        """Build a simple statistical model for the differential analysis"""
        print("Building simple statistical model for key recovery")
        
        # No neural network model - we'll use simple statistical methods instead
        self.model = {
            'key_bit_biases': np.zeros(64),  # Store biases for each key bit
            'trained': False
        }
        
        return self.model

    def train(self, epochs=30, batch_size=64, num_samples=5000):
        """Train the statistical model on differential pairs"""
        if self.model is None:
            self.build_model()
            
        print(f"Training statistical model on {num_samples} samples")
        
        # Generate training data
        X, y = self.generate_training_data(num_samples=num_samples)
        
        print(f"Generated {len(X)} samples with shape {X.shape}")
        print(f"Target shape: {y.shape}")
        
        # Simple statistical analysis of the relationship between input/output differences
        # and key bits (replacing neural network)
        
        # For each key bit, compute correlation with features
        for bit_pos in tqdm(range(64), desc="Analyzing key bit correlations"):
            # Extract the bit position from all targets
            bit_values = y[:, bit_pos]
            
            # Compute correlation between each feature and this key bit
            correlations = np.zeros(X.shape[1])
            for feat_pos in range(X.shape[1]):
                feature_values = X[:, feat_pos]
                # Simple correlation coefficient
                correlation = np.corrcoef(feature_values, bit_values)[0, 1]
                if not np.isnan(correlation):
                    correlations[feat_pos] = correlation
            
            # Store information about correlations for this key bit
            self.model['key_bit_biases'][bit_pos] = np.mean(bit_values)
        
        self.model['trained'] = True
        print("Statistical model training completed")
        
        # Return a mock history object for compatibility
        history = {
            'accuracy': [0.5 + 0.01 * epoch for epoch in range(5)],
            'val_accuracy': [0.5 + 0.008 * epoch for epoch in range(5)]
        }
        
        return history

    def predict(self, X):
        """Make predictions with the statistical model"""
        if not self.model or not self.model['trained']:
            raise ValueError("Model must be trained before predicting")
            
        num_samples = X.shape[0]
        predictions = np.zeros((num_samples, 64))
        
        # Use a simple heuristic instead of neural network
        # Set predictions based on the most common value for each key bit from training
        for i in range(64):
            # Use bias from training as a baseline prediction
            predictions[:, i] = self.model['key_bit_biases'][i]
        
        return predictions

    def attack(self, num_test_pairs=500, actual_key=None):
        """Attempt to recover key bits from the trained model
        
        Args:
            num_test_pairs: Number of differential pairs to use in the attack
            actual_key: The true key for validation (if known)
            
        Returns:
            predicted_key_bits: The predicted key bits
        """
        if self.model is None or not self.model.get('trained', False):
            raise ValueError("Model must be trained before attacking")
        
        print(f"Performing attack using {num_test_pairs} differential pairs")
        
        if actual_key:
            # For testing - create a cipher with the known key
            cipher = Present(actual_key, rounds=self.num_rounds)
            actual_last_round_key = cipher.roundkeys[-1]
            print(f"Actual last round key: {actual_last_round_key:016x}")
        else:
            # If no key is provided, we need to create one to generate test data
            print("No actual key provided. Generating random key for test data.")
            actual_key = bytes([random.randint(0, 255) for _ in range(16)])
            cipher = Present(actual_key, rounds=self.num_rounds)
            actual_last_round_key = cipher.roundkeys[-1]
            
        # Generate test differential pairs
        test_diffs = []
        
        for i in tqdm(range(num_test_pairs), desc="Generating test differential pairs"):
            try:
                # Similar to training data generation but with more structured differences
                plaintext1 = bytes([random.randint(0, 255) for _ in range(8)])
                
                if random.random() < 0.7:
                    # Single-bit difference
                    diff_position = random.randint(0, 63)
                    diff_mask = 1 << diff_position
                else:
                    # Byte-level difference
                    byte_pos = random.randint(0, 7)
                    byte_val = random.randint(1, 255)
                    diff_mask = byte_val << (byte_pos * 8)
                
                pt1_int = int.from_bytes(plaintext1, byteorder='big')
                pt2_int = pt1_int ^ diff_mask
                plaintext2 = pt2_int.to_bytes(8, byteorder='big')
                
                # Encrypt with the target cipher
                ciphertext1 = cipher.encrypt(plaintext1)
                ciphertext2 = cipher.encrypt(plaintext2)
                
                # Calculate differences
                input_diff = pt1_int ^ pt2_int
                output_diff = int.from_bytes(ciphertext1, byteorder='big') ^ int.from_bytes(ciphertext2, byteorder='big')
                
                # Convert to binary
                input_diff_binary = self._int_to_binary(input_diff, self.block_size)
                output_diff_binary = self._int_to_binary(output_diff, self.block_size)
                
                test_diffs.append(np.concatenate([input_diff_binary, output_diff_binary]))
                
            except Exception as e:
                print(f"Error generating test pair: {e}")
                continue
        
        print(f"Predicting key bits using {len(test_diffs)} test pairs")
        
        # Process in smaller batches
        batch_size = 50
        all_predictions = []
        
        for i in tqdm(range(0, len(test_diffs), batch_size), desc="Making predictions"):
            batch = np.array(test_diffs[i:i+batch_size])
            batch_predictions = self.predict(batch)
            all_predictions.append(batch_predictions)
        
        # Concatenate all batch predictions
        predictions = np.vstack(all_predictions)
        
        # Average predictions over all test pairs
        avg_prediction = np.mean(predictions, axis=0)
        
        # Convert probabilities to binary (0 or 1)
        predicted_key_bits = (avg_prediction > 0.5).astype(int)
        
        # Convert binary array back to integer
        predicted_key = self._binary_to_int(predicted_key_bits)
        
        print(f"Predicted last round key: {predicted_key:016x}")
        
        # Calculate success rate
        correct_bits = sum(1 for a, b in zip(self._int_to_binary(actual_last_round_key, 64), 
                                                predicted_key_bits) if a == b)
        success_rate = correct_bits / 64 * 100
        print(f"Success rate: {success_rate:.2f}% ({correct_bits}/64 bits correct)")
        
        # Analyze which bits were correctly recovered
        correct_positions = [i for i in range(64) 
                            if self._int_to_binary(actual_last_round_key, 64)[i] == predicted_key_bits[i]]
        print(f"Correct bit positions: {correct_positions}")
        
        # Check if certain byte positions were more accurate than others
        byte_accuracy = {}
        for byte_pos in range(8):
            start_bit = byte_pos * 8
            end_bit = start_bit + 8
            correct_in_byte = sum(1 for i in range(start_bit, end_bit) 
                                    if i in correct_positions)
            byte_accuracy[byte_pos] = correct_in_byte / 8 * 100
            print(f"Byte {byte_pos} accuracy: {byte_accuracy[byte_pos]:.2f}%")
            
        return predicted_key, success_rate, byte_accuracy

# Example usage without TensorFlow
def demonstrate_attack():
    print("Starting PRESENT cipher differential attack demo")

    # Set parameters for the demonstration
    rounds = 4  # Reduced rounds for better performance
    num_samples = 30000  # Reduced samples for quicker demonstration

    try:
        # Generate a random key for demonstration
        print("Generating random key...")
        key = os.urandom(16)  # 128-bit key
        print(f"Key (hex): {key.hex()}")
        
        # Create the attack object
        print(f"Creating attack object with {rounds} rounds...")
        attack = PresentDifferentialAttack(num_rounds=rounds)
        
        # Build and train the model
        print("Building and training model...")
        attack.build_model()
        history = attack.train(num_samples=num_samples)
        
        # Perform the attack
        print("Performing attack...")
        recovered_key, success_rate, byte_accuracy = attack.attack(actual_key=key, num_test_pairs=300)
        
        print("Attack completed successfully!")
        return recovered_key, success_rate, byte_accuracy, history
        
    except Exception as e:
        print(f"ERROR in demonstrate_attack: {e}")
        import traceback
        traceback.print_exc()
        return None, 0, {}, None

if __name__ == "__main__":
    print("Script starting")
    print("Running optimized PRESENT cipher differential attack")
    
    recovered_key, success_rate, byte_accuracy, history = demonstrate_attack()
    print("Script completed")

Script starting
Running optimized PRESENT cipher differential attack
Starting PRESENT cipher differential attack demo
Generating random key...
Key (hex): e0684f3bebcc6095575af010e1c86478
Creating attack object with 4 rounds...
Initializing PresentDifferentialAttack with 4 rounds
Building and training model...
Building simple statistical model for key recovery
Training statistical model on 30000 samples
Generating 30000 training samples...


Generating 128-bit roundkeys: 100%|██████████| 4/4 [00:00<00:00, 105517.08it/s]

Generating 128-bit roundkeys:   0%|          | 0/4 [00:00<?, ?it/s]